In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.datasets import cifar10

In [ ]:
physical_devices=tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

In [ ]:
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:
x_train=x_train/255.0
x_test=x_test/255.0

In [ ]:
x_test.shape

(10000, 32, 32, 3)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=42)

In [ ]:
class CNNBlock(layers.Layer):
  def __init__(self,output_channels,kernel_size=3):
    super(CNNBlock,self).__init__()
    self.conv=Conv2D(output_channels,kernel_size,padding='valid')
    self.bn=BatchNormalization()
    self.pooling=MaxPooling2D() # default pool size (2, 2), stride 2
  def call(self,input,training=False):
    x=self.conv(input)
    x=self.bn(x,training=training)
    x=tf.nn.relu(x)
    x=self.pooling(x)
    return x


In [ ]:
model=Sequential([CNNBlock(16),
                  CNNBlock(32),
                  CNNBlock(64),
                  layers.Flatten(),
                  layers.Dense(64,activation='relu'),
                  layers.Dense(10)])

In [ ]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(3e-4),
              metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=10,batch_size=64,validation_data=(x_val,y_val))

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.3387 - loss: 1.8742 - val_accuracy: 0.4914 - val_loss: 1.4184
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5259 - loss: 1.3313 - val_accuracy: 0.5555 - val_loss: 1.2588
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.5885 - loss: 1.1718 - val_accuracy: 0.5745 - val_loss: 1.1779
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6214 - loss: 1.0708 - val_accuracy: 0.6078 - val_loss: 1.1173
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6494 - loss: 0.9965 - val_accuracy: 0.5951 - val_loss: 1.1506
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6700 - loss: 0.9360 - val_accuracy: 0.6270 - val_loss: 1.0658
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6820 - loss: 0.9116 - val_accuracy: 0.6334 - val_loss: 1.0433
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6973 - loss: 0.8653 - val_accuracy: 0

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ cnn_block (CNNBlock)            │ (64, 15, 15, 16)       │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cnn_block_1 (CNNBlock)          │ (64, 6, 6, 32)         │         4,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cnn_block_2 (CNNBlock)          │ (64, 2, 2, 64)         │        18,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (64, 256)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, 64)               │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (64, 10)               │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,944 (480.25 KB)

 Trainable params: 40,906 (159.79 KB)

 Non-trainable params: 224 (896.00 B)

 Optimizer params: 81,814 (319.59 KB)

In [ ]:
model.evaluate(x_test,y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6274 - loss: 1.0707


[1.0666884183883667, 0.6310999989509583]

In [ ]:
def encoding_fn(text, labels):
    # Here, you define how you want to "encode" your text and labels
    return text, labels

def encoding_map(text, labels):
    encoded_text, encoded_labels = tf.py_function(
        func=encoding_fn,
        inp=[text, labels],
        Tout=(tf.int64, tf.int64)
    )
    encoded_text.set_shape([])
    encoded_labels.set_shape([])
    return encoded_text, encoded_labels


In [ ]:
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_val, y_val))
model.summary()
model.evaluate(x_test, y_test)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7367 - loss: 0.7626 - val_accuracy: 0.6427 - val_loss: 1.0474
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7462 - loss: 0.7358 - val_accuracy: 0.6347 - val_loss: 1.0708
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7551 - loss: 0.7063 - val_accuracy: 0.6653 - val_loss: 0.9861
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7642 - loss: 0.6843 - val_accuracy: 0.6547 - val_loss: 1.0448
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7675 - loss: 0.6732 - val_accuracy: 0.6651 - val_loss: 0.9971
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7762 - loss: 0.6488 - val_accuracy: 0.6691 - val_loss: 0.9804
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7861 - loss: 0.6307 - val_accuracy: 0.6721 - val_loss: 0.9935
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7868 - loss: 0.6167 - val_accuracy: 0.

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ cnn_block (CNNBlock)            │ (64, 15, 15, 16)       │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cnn_block_1 (CNNBlock)          │ (64, 6, 6, 32)         │         4,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cnn_block_2 (CNNBlock)          │ (64, 2, 2, 64)         │        18,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (64, 256)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, 64)               │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (64, 10)               │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,944 (480.25 KB)

 Trainable params: 40,906 (159.79 KB)

 Non-trainable params: 224 (896.00 B)

 Optimizer params: 81,814 (319.59 KB)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6647 - loss: 1.0302


[1.0323903560638428, 0.6679999828338623]